<a href="https://colab.research.google.com/github/isJC/Capstone_Project/blob/master/Segmenting_and_Clustering_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1 ----

In [0]:
#setup libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

print("Setup Complete")

Setup Complete


In [0]:
#Setup file
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(url, 'html.parser')

print("Setup File Request Complete")

Setup File Request Complete


In [0]:
#Scrape table
table = soup.find_all('table', class_="wikitable sortable")
df = pd.read_html(str(table))[0]

#drop rows of Borough with "Not assigned" values
df = df[df.Borough != 'Not assigned']

#Groupby Postcode and joined Neighourhood with the same postcode
df=df.groupby('Postcode').agg(lambda x: ' , '.join(set(x)))

df.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge , Malvern"
M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
M1E,Scarborough,"Guildwood , Morningside , West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [0]:
#Find rows that have "Not assigned" in Neighbourhood
df.loc[df['Neighbourhood'] == "Not assigned"]

,Borough,Neighbourhood
Postcode,,
M7A,Queen's Park,Not assigned


In [0]:
#replace with the Borough's name
df.replace("Not assigned", "Queen\'s Park", inplace = True)
#reset index
df1 = df.reset_index()
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [0]:
df1.shape

(103, 3)

# Part 2 ----

In [0]:
#Setup file
url = "http://cocl.us/Geospatial_data"

dfGeo = pd.read_csv(url)
dfGeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
#Rename column name
df1.rename(columns={"Postcode":"Postal Code"},inplace=True)
#Merge dataframes
df2 = df1.merge(dfGeo, on="Postal Code", how = 'left')
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#Part 3



Setup needed libraries and modules

In [0]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# # libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# # tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Setup Foursquare API credentials

In [118]:
CLIENT_ID = 'CLIENTID' # your Foursquare ID
CLIENT_SECRET = 'CLIENT SECRET' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENTID
CLIENT_SECRET:CLIENT SECRET


**Retrieve Toronto's Lat and Long**

In [0]:
#Retrieve Toronto Latitude and Longitude
address = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


**Searching for Chinese Restaurant within 100m radius**

In [80]:
search_query = 'Chinese Restaurant'
radius = 100
print("Looking for: " + search_query)

Looking for: Chinese Restaurant


In [0]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

In [82]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d889f10cad1b600399a7b25'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'primary': True,
      'shortName': 'Chinese'}],
    'hasPerk': False,
    'id': '4b2027b5f964a520f82d24e3',
    'location': {'address': '195 Dundas St W',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'crossStreet': 'at University Ave',
     'distance': 107,
     'formattedAddress': ['195 Dundas St W (at University Ave)',
      'Toronto ON M5G 1C7',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65492521335936,
       'lng': -79.38708916106386}],
     'lat': 43.65492521335936,
     'lng': -79.38708916106386,
     'postalCode': 'M5G 1C7',
     'state': 'ON'},
    'name': 'Hong Shing Chinese Restaurant',


**Convert results to dataframes**

In [83]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4b2027b5f964a520f82d24e3,195 Dundas St W,CA,Toronto,Canada,at University Ave,107,"[195 Dundas St W (at University Ave), Toronto ON M5G 1C7, Canada]","[{'label': 'display', 'lat': 43.65492521335936, 'lng': -79.38708916106386}]",43.654925,-79.387089,M5G 1C7,ON,Hong Shing Chinese Restaurant,v-1569234704,60327598
1,"[{'id': '4bf58dd8d48988d126941735', 'name': 'Government Building', 'pluralName': 'Government Buildings', 'shortName': 'Government', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_', 'suffix': '.png'}, 'primary': True}]",False,4f04779a02d5cce0cfc06151,"393 University Ave, Suite 1501",CA,Toronto,Canada,in University Centre,14,"[393 University Ave, Suite 1501 (in University Centre), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65402839343005, 'lng': -79.38736458967021}]",43.654028,-79.387365,NaN,ON,Chinese Visa Application Service Center,v-1569234704,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}]",False,4c69740b8d22c9284d42b745,56 Centre Ave.,CA,Toronto,Canada,NaN,96,"[56 Centre Ave., Toronto ON M5G 1R5, Canada]","[{'label': 'display', 'lat': 43.65483285234745, 'lng': -79.38720597193928}]",43.654833,-79.387206,M5G 1R5,ON,Wah Too Seafood Restaurant,v-1569234704,NaN
3,"[{'id': '4bf58dd8d48988d115941735', 'name': 'Middle Eastern Restaurant', 'pluralName': 'Middle Eastern Restaurants', 'shortName': 'Middle Eastern', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_', 'suffix': '.png'}, 'primary': True}]",False,4ddd83c788779c82beb061fc,199 Dundas St. W,CA,Toronto,Canada,at Centre Ave.,106,"[199 Dundas St. W (at Centre Ave.), Toronto ON M5g 1c8, Canada]","[{'label': 'display', 'lat': 43.65491647880546, 'lng': -79.38717216891472}]",43.654916,-79.387172,M5g 1c8,ON,Ali Baba's,v-1569234704,NaN


**Clean up dataframe. keeps only relevant columns**

In [84]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,CA,Toronto,Canada,at University Ave,107,"[195 Dundas St W (at University Ave), Toronto ON M5G 1C7, Canada]","[{'label': 'display', 'lat': 43.65492521335936, 'lng': -79.38708916106386}]",43.654925,-79.387089,M5G 1C7,ON,4b2027b5f964a520f82d24e3
1,Chinese Visa Application Service Center,Government Building,"393 University Ave, Suite 1501",CA,Toronto,Canada,in University Centre,14,"[393 University Ave, Suite 1501 (in University Centre), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65402839343005, 'lng': -79.38736458967021}]",43.654028,-79.387365,NaN,ON,4f04779a02d5cce0cfc06151
2,Wah Too Seafood Restaurant,Chinese Restaurant,56 Centre Ave.,CA,Toronto,Canada,NaN,96,"[56 Centre Ave., Toronto ON M5G 1R5, Canada]","[{'label': 'display', 'lat': 43.65483285234745, 'lng': -79.38720597193928}]",43.654833,-79.387206,M5G 1R5,ON,4c69740b8d22c9284d42b745
3,Ali Baba's,Middle Eastern Restaurant,199 Dundas St. W,CA,Toronto,Canada,at Centre Ave.,106,"[199 Dundas St. W (at Centre Ave.), Toronto ON M5g 1c8, Canada]","[{'label': 'display', 'lat': 43.65491647880546, 'lng': -79.38717216891472}]",43.654916,-79.387172,M5g 1c8,ON,4ddd83c788779c82beb061fc


**Filtered by Restaurant Names**

In [85]:
dataframe_filtered.name

0    Hong Shing Chinese Restaurant          
1    Chinese Visa Application Service Center
2    Wah Too Seafood Restaurant             
3    Ali Baba's                             
Name: name, dtype: object

In [86]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Toronto Chinese Restaurants

# add a red circle marker to represent the Toronto
#folium.features.CircleMarker(
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

**Compare with another Chinese Restaruant nearby**

In [0]:
venue_id = '4f04779a02d5cce0cfc06151' # Compare with another restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

In [91]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'venueRatingBlacklisted', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'attributes': {'groups': []},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1382635141,
  'height': 737,
  'id': '5269568511d259fd397a8bd8',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Foursquare for Android',
   'url': 'https://foursquare.com/download/#/android'},
  'suffix': '/24380_tRJAJqXHm6ylj_tEjZoP7HindomE-fWuGMDz2Y5Lies.jpg',
  'visibility': 'public',
  'width': 960},
 'canonicalUrl': 'https://foursquare.com/v/chinese-visa-application-service-center/4f04779a02d5cce0cfc06151',
 'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
    'suffix': '.png'},
   'id': '4bf58dd8d48988d126941735',
   'name': 'Government Building',
   'pluralName': 'Government Buildings',
   'primary': True,
   'shortName': 'Government'}],
 'colors': {'algoVersion': 3,
  'highlightColor': {'photoId': '5269568511d259fd397a8bd8',
   'value': -12040120}

**Check ratings of the restaurant**

In [92]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [94]:
venue_id = '4c69740b8d22c9284d42b745' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.1


In [95]:
result['response']['venue']['tips']['count']

11

In [96]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d88a040446ea600393f2221'},
 'response': {'tips': {'count': 11,
   'items': [{'agreeCount': 0,
     'canonicalUrl': 'https://foursquare.com/item/567c9ebc498ed7c004fff225',
     'createdAt': 1451007676,
     'disagreeCount': 0,
     'id': '567c9ebc498ed7c004fff225',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'photo': {'createdAt': 1451007680,
      'height': 1440,
      'id': '567c9ec0498ee31e8d2da41c',
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'suffix': '/10446912_YVX1R3iXY1qO1QBl3xY5wt72TLpDKhKar5DQsvj2u68.jpg',
      'visibility': 'public',
      'width': 1920},
     'photourl': 'https://fastly.4sqi.net/img/general/original/10446912_YVX1R3iXY1qO1QBl3xY5wt72TLpDKhKar5DQsvj2u68.jpg',
     'text': 'Spicy squid is fantastic. Dry chilli chicken is good. Everything we ordered was good.

In [97]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [98]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Spicy squid is fantastic. Dry chilli chicken is good. Everything we ordered was good. Super fast service. Like scary fast,0,0,567c9ebc498ed7c004fff225,Dominiquenotdom,Nomura,female,10446912


**Find user who provided a positive rating and his activities**

In [99]:
user_id = '10446912' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: Dominiquenotdom
Last Name: Nomura
Home City: Toronto, Canada


In [100]:
user_data['tips']

{'count': 203}

In [101]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,It’s all about the sauce! Chicken chimichanga and Mexican corn,0,0,5d87bbd834d80900069759e0
1,Unlimited refills on most drinks. Enchilada is pretty large for the price. Burrito supreme is massive. Cheese quesadilla for the kids. I’ll try the enchilada next time,0,0,5d828f88372b380006061f40
2,Call ahead for reservations. I waited over an hour for a table for 4 and people who came much later got a table before me. The food was delicious. They charge $4.50 tea/water. it’s a la carte.,0,0,5c7c1a3ecabcff0039b6f4a0
3,"Portions are large. We ordered à kids pizza, vegetarian pizza, steak calzone, bruschetta & carrot cake. The carrot cake is enough to feed 3 people. I'd order the pizza again and cake.",0,0,5bd91c46c66666003927ebb9
4,"Karage was good, gyozas were good. Had the pork ramen, it was ok.",1,0,5b19c6e0a22db70039825975
5,The cauliflower steak is good but next time I'll ask them not to put Olives on them as I'm not a fan,0,0,5a832073f0b4907b948d0574
6,No line up like bang bang and lots of flavours to choose from.,0,0,59e8080a2f97ec782aac5b79
7,A year later,0,0,59e2b54ca6031c4fcf1ed086
8,Taco salad was delicious. Corn tortillas for enchiladas and they were fantastic.,0,0,59488f78d48ec15bbe6aeabe
9,Desserts! This location has creme brûlée and pecan pie. Usually desserts suck at Mandarin :),0,0,5835eee7005a0f4a1aba2cc4


In [103]:
tip_id = '5b19c6e0a22db70039825975' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Kinton Ramen
{'address': '4026 Confederation', 'lat': 43.58539228095156, 'lng': -79.64570123284585, 'labeledLatLngs': [{'label': 'display', 'lat': 43.58539228095156, 'lng': -79.64570123284585}], 'postalCode': 'L5B', 'cc': 'CA', 'neighborhood': 'Cooksville', 'city': 'Mississauga', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['4026 Confederation', 'Mississauga ON L5B', 'Canada']}


In [104]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

,bio,firstName,gender,homeCity,id,lastName,lists.groups,photo.default,photo.prefix,photo.suffix,tips.count
0,,Gavin,male,Ontario,48903324,H.,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/4TOZCXDWGLZWLMOB.jpg,21
1,,Sandy,female,"Toronto, ON",71869926,Lai,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/71869926-4PO0LMEWK2Q4XGUM.jpg,2
2,,Neill,male,"Toronto, ON",62535903,Sturgess,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/UHXI3O53PD214LRE.jpg,0
3,,Michael,male,Québec,49392602,David,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/3IRV5P5PB1JOWKQQ.jpg,0
4,,Brian,male,Ontario,46265588,Ramsay,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/G52232PMPGDKTT4P.jpg,7
5,,Alex,male,Ontario,48261502,Rondini,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/YSEGSWE54Z0JTNO3.jpg,5
6,,Chris,male,Ontario,43508685,Melville,"[{'type': 'created', 'count': 2, 'items': []}]",NaN,https://fastly.4sqi.net/img/user/,/TCJL5CYIA35AFVY4.jpg,3
7,,Lisa,female,"Toronto, ON",68488740,Rondini,"[{'type': 'created', 'count': 2, 'items': []}]",True,https://fastly.4sqi.net/img/user/,/blank_girl.png,1
8,,Brian,none,"Toronto, ON",62909622,C,"[{'type': 'created', 'count': 2, 'items': []}]",True,https://fastly.4sqi.net/img/user/,/blank_boy.png,2
9,,Michael,male,"Toronto, ON",55790388,Ramsay,"[{'type': 'created', 'count': 2, 'items': []}]",True,https://fastly.4sqi.net/img/user/,/blank_boy.png,0


In [105]:
user_data

{'bio': '',
 'canonicalUrl': 'https://foursquare.com/user/10446912',
 'checkins': {'count': 1516, 'items': []},
 'contact': {},
 'firstName': 'Dominiquenotdom',
 'friends': {'count': 40,
  'groups': [{'count': 40,
    'items': [{'bio': '',
      'contact': {},
      'firstName': 'Gavin',
      'gender': 'male',
      'homeCity': 'Ontario',
      'id': '48903324',
      'lastName': 'H.',
      'lists': {'groups': [{'count': 2, 'items': [], 'type': 'created'}]},
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/4TOZCXDWGLZWLMOB.jpg'},
      'tips': {'count': 21}},
     {'bio': '',
      'contact': {},
      'firstName': 'Sandy',
      'gender': 'female',
      'homeCity': 'Toronto, ON',
      'id': '71869926',
      'lastName': 'Lai',
      'lists': {'groups': [{'count': 2, 'items': [], 'type': 'created'}]},
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/71869926-4PO0LMEWK2Q4XGUM.jpg'},
      'tips': {'count': 2}},
     {

In [106]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/user/300x300/4TOZCXDWGLZWLMOB.jpg')

**Find if there are any other nearby Chinese Restaurants**

In [0]:
latitude = 43.653963
longitude = -79.387207

 

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

In [0]:
import requests

In [109]:
results = requests.get(url).json()
'There are {} around restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 3 around restaurant.'

In [110]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4ad4c05ef964a520e2f620e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d18f941735',
    'name': 'Art Museum',
    'pluralName': 'Art Museums',
    'primary': True,
    'shortName': 'Art Museum'}],
  'id': '4ad4c05ef964a520e2f620e3',
  'location': {'address': '55 Centre Avenue',
   'cc': 'CA',
   'city': 'Toronto',
   'country': 'Canada',
   'crossStreet': 'University Ave. and Dundas St W.',
   'distance': 74,
   'formattedAddress': ['55 Centre Avenue (University Ave. and Dundas St W.)',
    'Toronto ON M5G 2H5',
    'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65439630500274,
     'lng': -79.38650010906946}],
   'lat': 43.65439630500274,
   'lng': -79.38650010906946,
   'postal

In [111]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Textile Museum of Canada,Art Museum,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,74,"[55 Centre Avenue (University Ave. and Dundas St W.), Toronto ON M5G 2H5, Canada]","[{'label': 'display', 'lat': 43.65439630500274, 'lng': -79.38650010906946}]",43.654396,-79.386500,M5G 2H5,ON,4ad4c05ef964a520e2f620e3
1,Starbucks,Coffee Shop,"439 University Ave,101",CA,Toronto,Canada,NaN,50,"[439 University Ave,101, Toronto ON M5G 2H6, Canada]","[{'label': 'display', 'lat': 43.654413, 'lng': -79.387201}]",43.654413,-79.387201,M5G 2H6,ON,5c5a865bbe7078002c14a735
2,Wah Too Seafood Restaurant,Chinese Restaurant,56 Centre Ave.,CA,Toronto,Canada,NaN,96,"[56 Centre Ave., Toronto ON M5G 1R5, Canada]","[{'label': 'display', 'lat': 43.65483285234745, 'lng': -79.38720597193928}]",43.654833,-79.387206,M5G 1R5,ON,4c69740b8d22c9284d42b745


In [112]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [113]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d88a15023bb8e003804d055'},
 'response': {'venues': []}}

In [0]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [115]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [0]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [117]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'